In [1]:
import requests
from bs4 import BeautifulSoup
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import jieba

headers  =  { "User-Agent":"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.2; WOW64; Trident/6.0; .NET4.0E; .NET4.0C; Tablet PC 2.0; .NET CLR 3.5.30729; .NET CLR 2.0.50727; .NET CLR 3.0.30729; McAfee; MAARJS)"
                 ,"Referer": "http://deviltrigger.oo.gd"}

res = requests.post(url = "https://deviltrigger.oo.gd/everything/trainingdata.php",headers = headers)

data = json.loads(res.text)['data']['ClassificationData']
print('新聞數量:',len(data))
print('前兩筆資料:',[data[x] for x in range(min(len(data),2))])

#將訓練資料中的title / description兩個欄位都串在一起做為訓練資料
corpus = [','.join(filter(None,(t['title'] ,t['description']))) for t in data]

新聞數量: 220
前兩筆資料: [{'id': '6553', 'title': '【採訪】家的色彩任你調！ Dulux 得利塗料首推VR 體驗塗刷樂趣', 'description': '有裝修經驗的人應該都會同意色彩絕對是左右居家氣氛與風格的一大要素， 以豐富電腦調色漆滿足屋主對居家色彩需求的Dulux 得利塗料今年正好是推出十周年，特地選在...', 'type': '居家'}, {'id': '6481', 'title': '20歲人妻自爆「有無限性慾」\u3000抖M看到雞雞抓狂...嘴巴被硬塞老二超爽！', 'description': '\n南港7條蛇／VC15文\n\n有看過片的男人都曉得，創意無限的SOD「都是真的」，且更強大的是', 'type': '正妹'}]


In [2]:
words = []
#tags = jieba.analyse.extract_tags('-->'.join([x for x in (td['title'],td['description']) if x])    , 10)
for i in [jieba.cut_for_search(w) for w in corpus]:    
    words.append('|'.join([w for w in i]))
#print(words)

corpus = words

vectorizer = TfidfVectorizer(max_df=0.5, max_features=1000, ngram_range=(1,1), min_df=2,use_idf=True)
#vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_

idfkeyword = dict(zip(vectorizer.get_feature_names(), idf))
#print(idfkeyword)

print('資料筆數: ',len(corpus))
print('idfkeyword total cnt(keyword 數量):',len(idfkeyword))
print('X.toarray()' , X.toarray())
#print(vectorizer.get_feature_names ())

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/xk/jzfb6tps7kz7xv_j3tjr0h7c0000gn/T/jieba.cache
Loading model cost 1.106 seconds.
Prefix dict has been built succesfully.


資料筆數:  220
idfkeyword total cnt(keyword 數量): 844
X.toarray() [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [3]:
#一般的kmean(一次性)
from sklearn.cluster import MiniBatchKMeans

k = int(len(corpus) * 0.8)
print('k:',k)
bestK = MiniBatchKMeans(n_clusters= k, init='k-means++', max_iter=10000, n_init=10, verbose=False)
bestK.fit(X)

score = bestK.score(X)
print('bestK Score:',score)

k: 176
bestK Score: -27.70549063263765


In [8]:
from sklearn.metrics.pairwise import pairwise_distances_argmin

print('center:','rows:',len(bestK.cluster_centers_),'cols:',len(bestK.cluster_centers_[0]),bestK.cluster_centers_)
print('size of labels:',len(bestK.labels_))

maplist = pairwise_distances_argmin(X,bestK.cluster_centers_)

print('maplist:',len(maplist),maplist)

#X_Grp = dict(zip(corpus))
#Center 雖然有向量內容，但不知道是X中的哪幾個??



center: rows: 176 cols: 844 [[ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.1161942 ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 ..., 
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]
 [ 0.         0.         0.        ...,  0.         0.         0.       ]]
size of labels: 220
maplist: 220 [123  50   5   5  79  26 150  80 142  70  63 113  30 118 107 134  64  54
 139 173  99 164  66  40 175 120  87   0   5   5  74  41  67  92 163  73
 102  97   5   5 162  42  46 151 161 104 116  91  94 171   5  33  84 109
  68   6  16   7  16 124   7 127 169 130  56 115 148  22  18  72 174  59
  53 156  48   7  69  58  12  78  38  49 168  85  57  39  22 110 154   5
   5  62  89  83  18  44  37  51  24 128  81 117  12  12 126 103  23 119
   5   5   5  11 157  43  43   5   5   5 131 106 

In [33]:
from sklearn.metrics.pairwise import pairwise_distances_argmin

xx = [[1,1],[2,2],[3,3]]
yy = [[1,1],[2,2],[3,4],[5,5]]
pairwise_distances_argmin(xx,yy)

array([0, 1, 2])

In [7]:
#輸出文章與Label的Dictionary
c = dict(zip(corpus,bestK.labels_))

c = sorted(c.items(), key=lambda x:x[1])

c

[('玩遊戲|太|認真|！|男生|全力|衝刺|「|向前|頂|」|力道|太|勇猛|！|女生|最後竟|濕|了|…|,|\n|文|／|金真露|\n|\n|昨天|的|直播|節目|讓|我|看|得欲罷|不能|R|，|真的|4|太|精采|了|！|所以|我|趕|緊來|找|看看',
  0),
 (' |經營|模擬|手機|遊戲|《|全民|百貨|》|封測|進行|中| |等級|上限|開放至| |LV| |28|,|\n|經營|模擬|手機|遊戲|《|全民|百貨|》|今|（|11|）|日雙|平台|封測|登場|，|等級|上限|開放至| |LV28|，|時尚|盛宴| |4F| |露天|派|對|開|放入|場|，|全新|精品|名牌|包開放|設櫃|以及|櫃姐|蘇菲亞|報到|，|娛樂|設施|、|各式|裝飾|道具|等|新品|上市|。|改版|期間|多項|限定|活動|進行|中|。|\xa0|&|nbs|...|\n',
  1),
 ('《|自|製|魔|物|獵|人|重|弩|》|可收|可擊發|完成|完成度|太高|啦|,|最近|關於|MH|的|消息|，|我|想|最|夯|的|就是|「|魔物|獵人| |世界|」|這款|遊戲|了|吧|，|顛覆過|往|大家|所有|的|認知|，|打臉|那些|認為|不會|平台|跨平台|的|玩家|，|這個|遊戲|從|釋出|消息|到試|玩|影片|出來|無|不|吸引|大家|的|注意|，|不過|阿漆|前面|會|這麼|有感|而發|，|...',
  2),
 ('【|模型|】|ALTER|《|偶像|大師| |姑娘|灰姑娘|女孩|》|鷺澤文香| |Bright| |Memories| |明年| |1| |月|預定|推出|,|\n|《|偶像|大師| |姑娘|灰姑娘|女孩|》|是|以|街|機用|模擬|遊戲|《|偶像|大師|》|世界|觀|的|元素|構成|，|開發|在|智慧|智慧型|電話|等|行動|裝置|上|的|一款|網頁|形式|免費|社交|網路|遊戲|，|由|DeNA|營運|和|提供|服務|、|運行|於|Mobage|平台|。|此|作品|動畫|於|2015|年|1|月|9|日|在|日本|首播|。|鷺澤文香|Brig|...|\n',
  3),
 ('手機|遊戲|《|盜賊|聯盟| |Gangster| |League|》|正式|開|放下|載|,|\n|盜賊|聯盟| |Gangster| |League| |在

http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
	
n_clusters : int, optional, default: 8
      中心向量的數量
      
max_iter : int, default: 300
      Maximum number of iterations of the k-means algorithm for a single run.
      
n_init : int, default: 10
      Number of time the k-means algorithm will be run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia.

tol : float, default: 1e-4
      Relative tolerance with regards to inertia to declare convergence
      
n_jobs : int
      The number of jobs to use for the computation. This works by computing each of the n_init runs in parallel.
If -1 all CPUs are used. If 1 is given, no parallel computing code is used at all, which is useful for debugging. For n_jobs below -1, (n_cpus + 1 + n_jobs) are used. Thus for n_jobs = -2, all CPUs but one are used.

random_state : integer or numpy.RandomState, optional
The generator used to initialize the centers. If an integer is given, it fixes the seed. Defaults to the global numpy random number generator.

verbose : int, default 0
      Verbosity mode.
copy_x : boolean, default True
      When pre-computing distances it is more numerically accurate to center the data first. If copy_x is True, then the original data is not modified. If False, the original data is modified, and put back before the function returns, but small numerical differences may be introduced by subtracting and then adding the data mean.

Attributes:	

cluster_centers_ : array, [n_clusters, n_features]
      Coordinates of cluster centers
labels_ : :
Labels of each point
inertia_ : float
       Sum of distances of samples to their closest cluster cente

In [18]:
bestK.__dict__

{'algorithm': 'auto',
 'batch_size': 100,
 'cluster_centers_': array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.1161942, ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        ..., 
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ]]),
 'compute_labels': True,
 'copy_x': True,
 'counts_': array([ 11,  15,  10,  23,   9, 413,  48,  56,  12,  30,  45,  30,  58,
         34,  38,  36,  35,  29,  36,  27,  30,  19,  41,  16,  42,  32,
         26,  20,  28,  41,  20,  19,  10,  18,  21,  13,  32,  12,  17,
         21,  12,  14,  19,  39,  25,  15,  21,  21,  20,  18,  13,  11,
 

In [4]:
from sklearn.cluster import MiniBatchKMeans

print([x for x in range(0,len(corpus))])

kmList = [MiniBatchKMeans(n_clusters= k, init='k-means++', max_iter=10000, n_init=10, verbose=False) for k in range(1,len(corpus))]

for k in kmList:
    k.fit(X)

kmDict = {k:k.score(X) for k in kmList}

bestK = max([k for k,v in kmDict.items()], key = lambda key:kmDict[key])
print('bestK Score:',kmDict[bestK])



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191]
bestK Score: 1.2212453270876722e-15
